# Spark Assignment
* Xuyan Xiao, xx2226
* Junhui Liao, jl4574

### Data Preparation

In [ ]:
from pyspark import SparkContext
sc =SparkContext()
# user and artist
rawUserArtistData = sc.textFile("user_artist_data.txt")

weights = [.1, .9]
seed = 42
rawUserArtistData, someOtherJunk = rawUserArtistData.randomSplit(weights, seed)
rawUserArtistData.cache()


# print rawUserArtistData.map(lambda l: float(l.split(" ")[0])).stats()
# print rawUserArtistData.map(lambda l: float(l.split(" ")[1])).stats()

In [ ]:
# user = rawUserArtistData.collect()[0]
# print user

In [ ]:
# artist data
rawArtistData = sc.textFile("artist_data.txt")

def lineSplit(artist):
    line = artist.split("\t")
    if len(line) < 2:
        return []
    else:
        try:
            return [(int(line[0]),line[1].strip())]
        except:
            return []

artistByID = dict(rawArtistData.flatMap(lambda l: lineSplit(l)).collect())

In [ ]:
# artist misspelled
rawArtistAlias = sc.textFile("artist_alias.txt")

def lineSplitAlias(artist):
    line = artist.split("\t",1)
    if line[0] == "":
        return []
    else:
        return [(int(line[0]),int(line[1]))]

artistAlias = rawArtistAlias.flatMap(lambda l: lineSplitAlias(l)).collectAsMap()


print artistByID[6803336]
print artistByID[1000010]

### build the first recommendation model

In [ ]:
from pyspark.mllib.recommendation import *
bArtistAlias = sc.broadcast(artistAlias)

def trainDataConstuct(line):
    userID, artistID, count = map(int,line.split(" "))
    finalArtistID = bArtistAlias.value.get(artistID,artistID)
    return Rating(userID, finalArtistID, count)

trainData = rawUserArtistData.map(lambda l: trainDataConstuct(l))
trainData.cache()

In [ ]:
model = ALS.trainImplicit(trainData, 10, 5, 0.01)

In [ ]:
ft = model.userFeatures().collect()
(ft[0][0],)+tuple([it for it in ft[0][1]])

In [ ]:
rawArtistsForUser = trainData.filter(lambda u: u.user == 1000002)

In [ ]:
t = rawArtistsForUser.collect()

In [ ]:
existingProducts = rawArtistsForUser.filter(lambda l: bArtistAlias.value.get(artistID,artistID))\
                                    .map(lambda m: )